# Checking radar line cross points


this time with the shapely intersection function

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import glob
import os
import sys
import time
import datetime as dt
import pandas as pd
import geopandas as gpd
import scipy as sp
from scipy import signal
from tqdm.notebook import tqdm

from shapely.geometry import Point, LineString, Polygon
import fiona

In [2]:
bed_gdf = gpd.read_file('/Volumes/arc_04/FIELD_DATA/K8621920/RES/PROCESSED_LINES/picked_bed.shp')

In [3]:
bed_gdf.head()

,CDP,x,y,DELAY,line_name,SEQnum,camp,geometry
0,100,-381564.6,-725999.7,8790,line7,1,KIS2,POINT (-381564.600 -725999.700)
1,101,-381563.4,-725998.3,8790,line7,1,KIS2,POINT (-381563.400 -725998.300)
2,102,-381561.6,-725996.1,8790,line7,1,KIS2,POINT (-381561.600 -725996.100)
3,103,-381559.5,-725993.6,8790,line7,1,KIS2,POINT (-381559.500 -725993.600)
4,104,-381557.4,-725991.1,8790,line7,1,KIS2,POINT (-381557.400 -725991.100)


# Convert all lines to a shapely line

In [4]:
linestrings = []
names = []

for line_name in bed_gdf.line_name.unique():
    linestrings.append( LineString(bed_gdf[bed_gdf.line_name==line_name].geometry.tolist() ) )
    names.append(line_name)

In [5]:
#put the lines in a geodataframe
lines_gdf = gpd.GeoDataFrame({'line_name':names},geometry=linestrings)
lines_gdf.head()

,line_name,geometry
0,line7,"LINESTRING (-381564.600 -725999.700, -381563.4..."
1,line9,"LINESTRING (-380741.900 -721788.100, -380743.5..."
2,line0,"LINESTRING (-377572.000 -734075.100, -377571.8..."
3,line10,"LINESTRING (-385757.300 -724442.800, -385756.3..."
4,line11,"LINESTRING (-384056.400 -720106.700, -384057.5..."


In [6]:
lines_gdf.set_index('line_name',inplace=True)

In [7]:
lines_gdf.index

Index(['line7', 'line9', 'line0', 'line10', 'line11', 'line12', 'line13',
       'line14', 'line1', 'line250', 'line2', 'line3', 'line4', 'line500',
       'line6', 'line7p25', 'line7p5', 'line7p75', 'line8', 'lineAPREScross',
       'lineAPRESdown', 'lineback2camp', 'linedownchan', 'lineend', 'linen250',
       'linen500', 'lineseis12', 'lineseis34', 'lineseis56', 'linestart',
       'lineupchan', 'line0KIS1', 'left02', 'left1011', 'left1213', 'left1415',
       'left35', 'left46', 'left79', 'left810', 'right1112', 'right1314',
       'right13', 'right24', 'right68', 'right910', 'line5'],
      dtype='object', name='line_name')

In [ ]:
line0s = []
line1s = []
intersection_point=[]

for line_name in lines_gdf.index.tolist():
    
    if not lines_gdf.loc['line6'].geometry.intersects(lines_gdf.loc['lineAPRESdown'].geometry)

In [17]:
lines_gdf.loc['line6'].intersects(lines_gdf.'lineapresdownchan'].geometry)

/Users/home/whitefar/miniconda3/envs/remote_sensing/lib/python3.7/site-packages/geopandas/geodataframe.py:546: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super(GeoDataFrame, self).__getitem__(key)


Series([], dtype: bool)

In [118]:
intersection_table = pd.DataFrame({})

for line_name in lines_gdf.index.tolist():

    intersection_table[line_name+'_'] = lines_gdf.geometry.intersection(lines_gdf.loc[line_name].geometry)

In [119]:
intersection_bool = pd.DataFrame({}) # boolean dataframe of whether theres an intersection

for line_name in lines_gdf.index.tolist():

    intersection_bool[line_name+'_'] = lines_gdf.geometry.intersects(lines_gdf.loc[line_name].geometry)

In [120]:
#to make this swifter we'll convert to numpy array
iv = intersection_table.values
ib = intersection_bool.values
np.fill_diagonal(ib,False) #set diagonals to nan

In [121]:
iv[~ib] = np.nan

In [122]:
intersection_table = pd.DataFrame(data = iv, columns= lines_gdf.index.tolist())

In [123]:
intersection_table['line_name'] = lines_gdf.index.tolist()
intersection_table.set_index('line_name',inplace=True)

In [125]:
intersection_table.line7.dropna()

line_name
lineAPRESdown    POINT (-380790.5614925373 -725148.5895522388)
lineupchan       POINT (-380743.1233597693 -725104.8605623648)
Name: line7, dtype: object

In [126]:
intersection_table.lineAPRESdown.dropna()

line_name
line7                 POINT (-380790.5614925373 -725148.5895522388)
line1                 POINT (-376887.8266365689 -731432.6374717833)
line2                 POINT (-377291.5200440529 -730091.2693832599)
line3                 POINT (-377762.1054704595 -728990.1148796498)
line4                 POINT (-378466.2854961831 -727714.8908396946)
line6                                POINT (-379814.83 -725890.096)
line7p25                              POINT (-380935.25 -725019.15)
line7p5           (POINT (-381138.6911392405 -724813.8540084389)...
lineAPREScross        POINT (-378873.4182741117 -726918.0197969544)
lineback2camp         POINT (-381104.4857916103 -724853.0419485791)
linedownchan                   POINT (-379680.8 -725990.8999999999)
lineupchan            POINT (-381007.2834661354 -724943.2557768924)
line5                 POINT (-379060.4531914893 -726579.6468085106)
Name: lineAPRESdown, dtype: object

In [138]:
intersection_table.to_pickle('/Users/home/whitefar/DATA/FIELD_ANT_19/POST_FIELD/RADAR_MAP/lines_intersectionpoints.pkl')

In [144]:
intersection_bool = pd.DataFrame(data = ib, columns= lines_gdf.index.tolist())
intersection_bool['line_name'] = lines_gdf.index.tolist()
intersection_bool.set_index('line_name',inplace=True)
intersection_bool.line7.head()

line_name
line7     False
line9     False
line0     False
line10    False
line11    False
Name: line7, dtype: bool

In [145]:
#which lines intersect with which
intersection_bool.to_pickle('/Users/home/whitefar/DATA/FIELD_ANT_19/POST_FIELD/RADAR_MAP/lines_intersects_boolean.pkl')

In [105]:
from shapely.ops import nearest_points

In [115]:
nearest_points(lines_gdf.loc['line7'].geometry,lines_gdf.loc['line8'].geometry)

(<shapely.geometry.point.Point at 0x7f7bfa2f4a20>,
 <shapely.geometry.point.Point at 0x7f7bf9fc8b70>)

# Using nearest_points make the same table but with tuples of points from the lines

In [130]:
listofrows = []
for line_name0 in lines_gdf.index.tolist():
    row = []
    for line_name1 in lines_gdf.index.tolist():        
        row.append( nearest_points(lines_gdf.loc[line_name0].geometry,lines_gdf.loc[line_name1].geometry) )
    listofrows.append(row)

In [135]:
listofrows_df = pd.DataFrame(data=listofrows, columns = lines_gdf.index.tolist())
listofrows_df['line_name'] = lines_gdf.index.tolist()
listofrows_df.set_index('line_name',inplace=True)

In [136]:
listofrows_df.head()

,line7,line9,line0,line10,line11,line12,line13,line14,line1,line250,...,left46,left79,left810,right1112,right1314,right13,right24,right68,right910,line5
line_name,,,,,,,,,,,,,,,,,,,,,
line7,"(POINT (-381564.6 -725999.7), POINT (-381564.6...","(POINT (-379471.7 -723757.5), POINT (-381065.4...","(POINT (-381564.6 -725999.7), POINT (-375351 -...","(POINT (-379506.8 -723793.7), POINT (-382482.7...","(POINT (-380261.2 -724574.5), POINT (-384405.8...","(POINT (-381103.1 -725464.4), POINT (-386882.2...","(POINT (-381075.9 -725435.5), POINT (-387797.9...","(POINT (-381075.9 -725435.5), POINT (-389217.9...","(POINT (-381564.6 -725999.7), POINT (-376491 -...","(POINT (-378995.7 -723264.8), POINT (-334455 -...",...,"(POINT (-378995.7 -723264.8), POINT (-377898 -...",(POINT (-379011.9779345089 -723280.4098740555)...,"(POINT (-379047.5 -723315.1), POINT (-379938.4...","(POINT (-381564.6 -725999.7), POINT (-387514.3...","(POINT (-381564.6 -725999.7), POINT (-391106.5...","(POINT (-381564.6 -725999.7), POINT (-379422.6...","(POINT (-381564.6 -725999.7), POINT (-379981.4...","(POINT (-381564.6 -725999.7), POINT (-382001.1...","(POINT (-381564.6 -725999.7), POINT (-383945.9...","(POINT (-379749.6 -724075.7), POINT (-378099.3..."
line9,"(POINT (-381065.4 -722134), POINT (-379471.7 -...","(POINT (-380741.9 -721788.1), POINT (-380741.9...","(POINT (-382203.7 -723326.1), POINT (-374781.9...",(POINT (-381259.6395749518 -722307.6349224534)...,"(POINT (-381651.1 -722705.4), POINT (-384218.5...","(POINT (-383950.1 -725092.2), POINT (-388050.6...","(POINT (-383790.8 -724929.7), POINT (-388879.4...","(POINT (-382914.8 -724011.1), POINT (-389399.3...","(POINT (-382196.6 -723318.9), POINT (-375437.3...","(POINT (-380741.9 -721788.1), POINT (-334455 -...",...,"(POINT (-380741.9 -721788.1), POINT (-377898 -...","(POINT (-380741.9 -721788.1), POINT (-380735.7...","(POINT (-380864.9949999999 -721900.3025), POIN...","(POINT (-383950.1 -725092.2), POINT (-387514.3...","(POINT (-383950.1 -725092.2), POINT (-391106.5...","(POINT (-383950.1 -725092.2), POINT (-379422.6...","(POINT (-383950.1 -725092.2), POINT (-379981.4...","(POINT (-383855.6 -724996.7), POINT (-383017.3...",(POINT (-383937.7500000002 -725080.0500000002)...,"(POINT (-382195.2 -723317.4), POINT (-378886.8..."
line0,"(POINT (-375351 -731659.2), POINT (-381564.6 -...","(POINT (-374781.9 -731037.1), POINT (-382203.7...","(POINT (-377572 -734075.1), POINT (-377572 -73...","(POINT (-374781.9 -731037.1), POINT (-383697.7...","(POINT (-374781.9 -731037.1), POINT (-384760 -...","(POINT (-375841.8 -732170.2), POINT (-387395.1...","(POINT (-374781.9 -731037.1), POINT (-387797.9...","(POINT (-374781.9 -731037.1), POINT (-389207.5...","(POINT (-375930.6 -732263), POINT (-376855 -73...","(POINT (-374781.9 -731037.1), POINT (-334455 -...",...,"(POINT (-374781.9 -731037.1), POINT (-376819.9...","(POINT (-374781.9 -731037.1), POINT (-379027.1...","(POINT (-374781.9 -731037.1), POINT (-379938.4...","(POINT (-377128.4 -733566), POINT (-387514.3 -...","(POINT (-377538.9 -734007.1), POINT (-391106.5...","(POINT (-377306.6 -733755.7), POINT (-378261.1...","(POINT (-376973.5 -733406.3), POINT (-378801.5...","(POINT (-375853 -732181.7), POINT (-381116 -72...","(POINT (-375945.7 -732279.5), POINT (-383945.9...","(POINT (-375346 -731653.8), POINT (-379840.2 -..."
line10,"(POINT (-382482.7 -721042.4), POINT (-379506.8...","(POINT (-382482.7 -721042.4), POINT (-381259.6...","(POINT (-383697.7 -722250.3), POINT (-374781.9...","(POINT (-385757.3 -724442.8), POINT (-385757.3...","(POINT (-383482.471464966 -722011.4989808893),...","(POINT (-384951.8 -723551), POINT (-387560 -72...","(POINT (-384456.1 -723026.1), POINT (-388300.9...","(POINT (-384456.1 -723026.1), POINT (-389686.5...","(POINT (-383691.5 -722243.9), POINT (-375541.7...","(POINT (-382482.7 -721042.4), POINT (-334455 -...",...,"(POINT (-382482.7 -721042.4), POINT (-377898 -...","(POINT (-382482.7 -721042.4), POINT (-380747.9...","(POINT (-382482.7 -721042.4), POINT 

In [137]:
# This file has the closest two points of each 2 lines 
listofrows_df.to_pickle('/Users/home/whitefar/DATA/FIELD_ANT_19/POST_FIELD/RADAR_MAP/nearest_points_table.pkl')

# Now run through the nearest points and get errors.

## first mark 'intersection'=True on the bed_df file

In [8]:
nearest_points_raw = pd.read_pickle('/Users/home/whitefar/DATA/FIELD_ANT_19/POST_FIELD/RADAR_MAP/nearest_points_table.pkl')
intersection_bool = pd.read_pickle('/Users/home/whitefar/DATA/FIELD_ANT_19/POST_FIELD/RADAR_MAP/lines_intersects_boolean.pkl')

In [21]:
print(nearest_points_raw.loc['line7','lineAPRESdown'][0].wkt,nearest_points_raw.loc['line7','lineAPRESdown'][1].wkt)

POINT (-380790.5614925373 -725148.5895522388) POINT (-380790.5614925373 -725148.5895522388)


In [11]:
#to make this swifter we'll convert to numpy array
nearp = nearest_points_raw.values
ib = intersection_bool.values

In [12]:
nearp[~ib] = np.nan

In [16]:
# This dataframe near_points_at_intersection has the closest two points of each 2 lines, just for the lines which intersect
near_points_at_intersection = pd.DataFrame(data = nearp, columns= lines_gdf.index.tolist())
near_points_at_intersection['line_name'] = lines_gdf.index.tolist()
near_points_at_intersection.set_index('line_name',inplace=True)
near_points_at_intersection.line7.dropna()

line_name
lineAPRESdown    (POINT (-380790.5614925373 -725148.5895522388)...
lineupchan       (POINT (-380743.1233597693 -725104.8605623648)...
Name: line7, dtype: object

In [32]:
ip_values = intersecting_points.values
ip_list[~np.isnan(ip_list)]

TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [49]:
#a function which inputs point and outputs delay using bed_gdf
a = Point(-380790.5614925373, -725148.5895522388)
bed_gdf[(bed_gdf.x==a.x) & (bed_gdf.y==a.y)]

IndexError: too many indices for array

Empty GeoDataFrame
Columns: [CDP, x, y, DELAY, line_name, SEQnum, camp, geometry]
Index: []

In [64]:
np.argwhere((bed_gdf.geometry==a).to_numpy())

array([], shape=(0, 1), dtype=int64)

In [65]:
(bed_gdf.geometry==a).to_numpy()

array([False, False, False, ..., False, False, False])

In [73]:
bed_gdf[bed_gdf.line_name=='line7'].geometry.x.tolist()

[-381564.6,
 -381563.4,
 -381561.6,
 -381559.5,
 -381557.4,
 -381555.0,
 -381553.8,
 -381551.4,
 -381550.0,
 -381547.4,
 -381546.0,
 -381543.4,
 -381542.2,
 -381539.2,
 -381538.0,
 -381535.3,
 -381534.0,
 -381531.4,
 -381530.1,
 -381527.6,
 -381526.3,
 -381524.9,
 -381522.2,
 -381520.7,
 -381517.9,
 -381516.7,
 -381514.0,
 -381512.5,
 -381511.2,
 -381508.8,
 -381507.5,
 -381505.0,
 -381503.7,
 -381501.2,
 -381500.0,
 -381497.5,
 -381496.1,
 -381494.9,
 -381492.5,
 -381491.1,
 -381488.7,
 -381487.2,
 -381486.1,
 -381483.7,
 -381482.4,
 -381480.1,
 -381478.9,
 -381476.6,
 -381475.4,
 -381473.0,
 -381471.9,
 -381469.5,
 -381468.3,
 -381465.8,
 -381464.7,
 -381462.1,
 -381460.9,
 -381459.6,
 -381457.2,
 -381455.9,
 -381453.1,
 -381451.8,
 -381449.3,
 -381447.9,
 -381445.1,
 -381443.6,
 -381442.3,
 -381439.6,
 -381438.1,
 -381435.4,
 -381433.9,
 -381431.1,
 -381429.8,
 -381427.0,
 -381425.7,
 -381422.7,
 -381421.2,
 -381418.5,
 -381416.9,
 -381415.5,
 -381412.6,
 -381411.2,
 -381408.3,
 -38